In [2]:
!python -m spacy download de_core_news_sm
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 58.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's depende

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datasets
import spacy
import torch.utils
import torch.utils.data
import tqdm
import random

device = torch.device("cuda" if torch.cpu.is_available() else "cpu")
dataset = datasets.load_dataset("bentrevett/multi30k")

en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"
unk_token = "<unk>"

train_data, valid_data, test_data = (
  dataset["train"],
  dataset["validation"],
  dataset["test"],
)

def tokenize_data(data, en_nlp, de_nlp, sos_token, eos_token):
  en_tokens = [token.text.lower() for token in en_nlp.tokenizer(data["en"])]
  de_tokens = [token.text.lower() for token in de_nlp.tokenizer(data["de"])]
  en_tokens = [sos_token] + en_tokens + [eos_token]
  de_tokens = [sos_token] + de_tokens + [eos_token]
  return {"en_tokens": en_tokens, "de_tokens": de_tokens}

fn_kwargs = {
  "en_nlp": en_nlp,
  "de_nlp": de_nlp,
  "sos_token": sos_token,
  "eos_token": eos_token,
}

train_data = train_data.map(tokenize_data, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_data, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_data, fn_kwargs=fn_kwargs)

def build_vocab(token_lists, min_freq=1):
  s = set([token for tokens in token_lists for token in tokens])
  vocab = {}
  vocab[unk_token] = len(vocab)
  vocab[pad_token] = len(vocab)
  vocab[sos_token] = len(vocab)
  vocab[eos_token] = len(vocab)
  for token in s:
    if token not in vocab:
      vocab[token] = len(vocab)
  return vocab

en_vocab = build_vocab(train_data["en_tokens"])
de_vocab = build_vocab(train_data["de_tokens"])

def text_to_ids(texts, vocab):
  return [[vocab.get(token, vocab[unk_token]) for token in text] for text in texts]

en_ids = text_to_ids(train_data["en_tokens"], en_vocab)
de_ids = text_to_ids(train_data["de_tokens"], de_vocab)




/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, src, tgt, src_vocab, tgt_vocab, pad_token='<pad>'):
        self.src = src
        self.tgt = tgt
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.pad_token = pad_token
        self.src_max_length = max(len(seq) for seq in src)
        self.tgt_max_length = max(len(seq) for seq in tgt)

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        src_ids = self.src[idx]
        tgt_ids = self.tgt[idx]
        # Pad the source and target sequences
        src_ids = src_ids + [self.src_vocab[self.pad_token]] * (self.src_max_length - len(src_ids))
        tgt_ids = tgt_ids + [self.tgt_vocab[self.pad_token]] * (self.tgt_max_length - len(tgt_ids))

        # Convert lists to tensors
        src_tensor = torch.tensor(src_ids, dtype=torch.long)
        tgt_tensor = torch.tensor(tgt_ids, dtype=torch.long)

        return src_tensor, tgt_tensor


In [40]:
# model
class Encoder(nn.Module):
  def __init__(self, input_size, embed_size, hidden_size, n_layers, dropout_rate):
    super(Encoder, self).__init__()
    self.embedding = nn.Embedding(input_size, embed_size)
    self.bigru = nn.GRU(embed_size, hidden_size, n_layers, bidirectional=True, dropout=dropout_rate)
    self.fc = nn.Linear(hidden_size*2, hidden_size)
    self.dropout = nn.Dropout(dropout_rate)
  def forward(self, input):
    # input: (seq_length, batch_size)
    embedded = self.dropout(self.embedding(input)) # (seq_length, batch_size, embed_size)
    outputs, hidden = self.bigru(embedded)
    # output: (seq_length, batch_size, hidden_size*2)
    # hidden: (n_layers * n_directions, batch_size, hidden_size)
    hidden_cat = torch.cat([hidden[-2,:,:], hidden[-1,:,:]], dim=1)
    # hidden_cat: (batch_size, hidden_size*2) - n_layers set to 1 here for simplicity
    hidden_out = F.relu(self.fc(hidden_cat))
    # hidden_out: (batch_size, hidden_size)
    return outputs, hidden_out

class BahdanauAttention(nn.Module):
  def __init__(self, hidden_size):
    super(BahdanauAttention, self).__init__()
    self.wa = nn.Linear(hidden_size, hidden_size, bias=False)
    self.ua = nn.Linear(hidden_size * 2, hidden_size, bias=False)
    self.va = nn.Linear(hidden_size, 1, bias=False)
  def forward(self, hidden, encoder_outputs):
    # hidden: (batch_size, hidden_size)
    # encoder_outputs: (seq_length, batch_size, hidden_size*2)
    seq_len = encoder_outputs.shape[0]
    hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch_size, seq_length, hidden_size)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)  # (batch_size, seq_length, hidden_size * 2)
    # calculate alignment scores
    energy = F.relu(self.wa(hidden) + self.ua(encoder_outputs))  # (batch_size, seq_length, hidden_size)
    scores = self.va(energy)  # (batch_size, seq_length, 1)
    scores = scores.squeeze(2)  # (batch_size, seq_length)

    # calculate the attention weights (prob) from alignment scores
    attn_weights = F.softmax(scores, dim=-1)  # (batch_size, seq_length)

    # calculate context vector
    context_vector = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)  # (batch_size, 1, hidden_size*2)
    #print(context_vector.shape, attn_weights.shape, hidden.shape)
    # context_vector: (batch_size, 1, hidden_size * 2)
    # alphas: (batch_size, seq_length)
    return context_vector, attn_weights

class Decoder(nn.Module):
  def __init__(self, output_size, embed_size, hidden_size, n_layers, dropout_rate):
    super(Decoder, self).__init__()
    self.output_size = output_size
    self.embedding = nn.Embedding(output_size, embed_size)
    self.gru = nn.GRU(embed_size + hidden_size*2, hidden_size, n_layers, bidirectional=False, dropout=dropout_rate)
    self.attention = BahdanauAttention(hidden_size)
    self.fc = nn.Linear(embed_size + hidden_size + hidden_size*2, output_size)
    self.dropout = nn.Dropout(dropout_rate)
  def forward(self, input, hidden, encoder_outputs):
    # input: (batch_size, 1)
    # hidden: (batch_size, hidden_size)
    # encoder_outputs: (seq_length, batch_size, hidden_size*2)
    embedded = self.dropout(self.embedding(input.unsqueeze(0))) # (1, batch_size, embed_size)
    context_vector, attn_weights = self.attention(hidden, encoder_outputs)
    # context_vector: (batch_size, 1, hidden_size*2)
    # attn_weights: (batch_size, seq_length)
    context_vector = context_vector.permute(1, 0, 2) # (1, batch_size, hidden_size*2)
    #print(context_vector.shape, embedded.shape)
    #print(input.shape)
    gru_input = torch.cat([embedded, context_vector], dim=2) # (1, batch_size, embed_size + hidden_size*2)
    output, hidden = self.gru(gru_input, hidden.unsqueeze(0))
    # output: (1, batch_size, hidden_size)
    # hidden: (1, batch_size, hidden_size)
    embedded = embedded.squeeze(0) # (batch_size, embed_size)
    output = output.squeeze(0) # (batch_size, hidden_size)
    context_vector = context_vector.squeeze(0) # (batch_size, hidden_size*2)
    pred = self.fc(torch.cat([embedded, output, context_vector], dim=1)) # (batch_size, output_size)
    hidden = hidden.squeeze(0) # (batch_size, hidden_size)
    return pred, hidden, attn_weights

class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, source, target, teacher_force_ratio=0.5):
    # source: (source_seq_length, batch_size)
    # target: (target_seq_length, batch_size)
    target_seq_length, batch_size = target.shape
    target_vocab_size = self.decoder.output_size
    outputs = torch.zeros((target_seq_length, batch_size, target_vocab_size)).to(device)
    encoder_outputs, hidden = self.encoder(source)
    # encoder_outputs: (source_seq_length, batch_size, hidden_size*2)
    # hidden: (batch_size, hidden_size)
    input = target[0, :] # (batch_size,) - initial input is sos token
    for t in range(1, target_seq_length):
      output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
      # output: (batch_size, output_size)
      # hidden: (batch_size, hidden_size)
      outputs[t] = output
      top1 = output.argmax(dim=1)
      input = target[t] if random.random() < teacher_force_ratio else top1
    return outputs

In [49]:
def train_fn(model, dataloader, optimizer, criterion, clip=4.0):
  model.train()
  total_loss = 0

  for idx, (src, tgt) in enumerate(tqdm.tqdm(dataloader, total=len(dataloader), position=0, leave=True)):
    src, tgt = src.to(device), tgt.to(device)
    src, tgt = src.T, tgt.T
    # src: (src_seq_length, batch_size)
    # tgt: (tgt_seq_length, batch_size)
    optimizer.zero_grad()
    output = model(src, tgt) # (target_seq_length, batch_size, output_size)
    output = output.view(-1, output.shape[-1])  # (target_seq_length*batch_size, output_size)
    tgt = tgt.contiguous().view(-1)  # (target_seq_length*batch_size)
    loss = criterion(output, tgt)
    total_loss += loss.item()
    loss.backward()
    output = output.argmax(dim=-1)
    nn.utils.clip_grad_norm_(model.parameters(), clip) # gradient clipping
    optimizer.step()
    if (idx + 1) % 50 == 0:
      print(f"loss: {loss.item()}")

  return total_loss / len(dataloader)


In [47]:
# train
INPUT_DIM = len(en_vocab)
OUTPUT_DIM = len(de_vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 1
DROPOUT = 0.5

enc = Encoder(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(device)
dec = Decoder(OUTPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(device)
model = Seq2Seq(enc, dec).to(device)

BATCH_SIZE = 64
train_dataset = TranslationDataset(en_ids, de_ids, en_vocab, de_vocab)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=en_vocab[pad_token])

# test train
num_epochs = 10
for epoch in range(num_epochs):
  epoch_loss = train_fn(model, train_loader, optimizer, criterion)
  print(f"epoch {epoch+1}, loss: {epoch_loss}")

 11%|█         | 50/454 [00:38<05:28,  1.23it/s]

loss: 5.3592329025268555


 22%|██▏       | 100/454 [01:15<04:49,  1.22it/s]

loss: 4.905766487121582


 33%|███▎      | 150/454 [01:53<04:24,  1.15it/s]

...

loss: 2.6762852668762207


 99%|█████████▉| 450/454 [05:41<00:03,  1.21it/s]

loss: 1.8085353374481201


100%|██████████| 454/454 [05:44<00:00,  1.32it/s]

epoch 10, loss: 2.039632056253072


In [48]:
for epoch in range(50):
  epoch_loss = train_fn(model, train_loader, optimizer, criterion)
  print(f"epoch {epoch+1}, loss: {epoch_loss}")

 11%|█         | 50/454 [00:38<05:43,  1.18it/s]

loss: 1.9428837299346924


 22%|██▏       | 100/454 [01:16<04:58,  1.19it/s]

loss: 1.923464298248291


 33%|███▎      | 150/454 [01:54<04:18,  1.18it/s]

...

loss: 1.6989742517471313


 88%|████████▊ | 400/454 [05:04<00:44,  1.23it/s]

loss: 1.837982177734375


 96%|█████████▋| 438/454 [05:33<00:12,  1.31it/s]


KeyboardInterrupt: 